Assignment:

In this assignment, you'll continue working with the house prices data. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

- Load the houseprices data from Thinkful's database.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.naive_bayes import BernoulliNB
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse,rmse
from sqlalchemy import create_engine

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
# Connect to the data base:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format( postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices', con=engine)
engine.dispose()

In [3]:
# Look at the dataset:
df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.000,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


- Reimplement your model from the previous checkpoint.

In [4]:
# Convert categorical columns to numeric by using dummies:
categorical = df.select_dtypes(include=['object'])
dummies = pd.get_dummies(categorical, drop_first=True)
df = df.drop(categorical, axis=1)
df = pd.concat([df,dummies], axis=1)

In [5]:
# Now we build a simple linear regression model and estimate it using OLS:

# Y is the target variable:
Y = df['saleprice']
# X is the feature set:
X = df[['overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf', 'exterqual_TA', 'kitchenqual_TA']]

In [6]:
# Split dataset:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}".format(X_test.shape[0]))

The number of observations in training set is 1168
The number of observations in test set is 292


- Try OLS, Lasso, Ridge, and ElasticNet regression using the same model specification. This time, you need to do k-fold cross-validation to choose the best hyperparameter values for your models. Scikit-learn has RidgeCV, LassoCV, and ElasticNetCV that you can utilize to do this. Which model is the best? Why?

In [7]:
# We fit an OLS model using sklearn:
lrm = LinearRegression()
lrm.fit(X_train, y_train)


# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in the training set is: 0.7653043702630363
-----Test set statistics-----
R-squared of the model in the test set is: 0.7744461385990857
Mean absolute error of the prediction is: 25427.91243226023
Mean squared error of the prediction is: 1514301879.3380787
Root mean squared error of the prediction is: 38914.03190801589
Mean absolute percentage error of the prediction is: 15.3407185525745


As we see, the R-squared of the model in the training set is 0.77; also, R-squared of the model in the test set is 0.77.


In [8]:
# ???? I dont know how to use k-fold cross-validation to choose the best hyperparameter values for your models?????
kf = KFold(n_splits=2)
kf.get_n_splits(X)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)    

TRAIN: [ 730  731  732  733  734  735  736  737  738  739  740  741  742  743
  744  745  746  747  748  749  750  751  752  753  754  755  756  757
  758  759  760  761  762  763  764  765  766  767  768  769  770  771
  772  773  774  775  776  777  778  779  780  781  782  783  784  785
  786  787  788  789  790  791  792  793  794  795  796  797  798  799
  800  801  802  803  804  805  806  807  808  809  810  811  812  813
  814  815  816  817  818  819  820  821  822  823  824  825  826  827
  828  829  830  831  832  833  834  835  836  837  838  839  840  841
  842  843  844  845  846  847  848  849  850  851  852  853  854  855
  856  857  858  859  860  861  862  863  864  865  866  867  868  869
  870  871  872  873  874  875  876  877  878  879  880  881  882  883
  884  885  886  887  888  889  890  891  892  893  894  895  896  897
  898  899  900  901  902  903  904  905  906  907  908  909  910  911
  912  913  914  915  916  917  918  919  920  921  922  923  924  925

In [9]:
# Finding best alpha:
# ????? Also , i do it this way to find alpha, but I dont know how to find best alpha from result???? 
bnb = BernoulliNB()
y_pred = bnb.fit(X_train, y_train).predict(X_train)
print('Train Array: {}'.format(cross_val_score(bnb, X_train, y_train, cv=5)))

y_pred = bnb.fit(X_test, y_test).predict(X_test)
print('Test Array: {}'.format(cross_val_score(bnb, X_test, y_test, cv=5)))

Train Array: [0.00476948 0.01219512 0.01369863 0.03488372 0.03278689]
Test Array: [0.00454545 0.02       0.13333333 0.2        0.5       ]


In [10]:
# LassoCV regression:

#???? how can i know give alpha numbers like below:???????
alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)] 

lassoregr = LassoCV(alphas=alphas, cv=5) #?? cv what does means???
lassoregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lassoregr.predict(X_train)
y_preds_test = lassoregr.predict(X_test)

print("Best alpha value is: {}".format(lassoregr.alpha_))
print("R-squared of the model on the training set is: {}".format(lassoregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(lassoregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


Best alpha value is: 1e-10
R-squared of the model on the training set is: 0.7653043702630362
-----Test set statistics-----
R-squared of the model on the test set is: 0.7744461385990783
Mean absolute error of the prediction is: 25427.91243226072
Mean squared error of the prediction is: 1514301879.3381288
Root mean squared error of the prediction is: 38914.031908016535
Mean absolute percentage error of the prediction is: 15.340718552574847


As we see, the R-squared of the model in the training set is 0.77; also, R-squared of the model in the test set is 0.77.


In [11]:
# Fitting a ridge regression model. 
ridgeregr = RidgeCV(alphas=alphas, cv=5)
ridgeregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridgeregr.predict(X_train)
y_preds_test = ridgeregr.predict(X_test)

print("Best alpha value is: {}".format(ridgeregr.alpha_))
print("R-squared of the model on the training set is: {}".format(ridgeregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridgeregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


Best alpha value is: 10.0
R-squared of the model on the training set is: 0.7652841680051667
-----Test set statistics-----
R-squared of the model on the test set is: 0.7747510438155123
Mean absolute error of the prediction is: 25350.760203657755
Mean squared error of the prediction is: 1512254835.9428246
Root mean squared error of the prediction is: 38887.72088902646
Mean absolute percentage error of the prediction is: 15.290493798195367


As we see, the R-squared of the model in the training set is 0.77; also, R-squared of the model in the test set is 0.77.


In [12]:
# ElasticNet regression:
elasticregr = ElasticNetCV(alphas=alphas, cv=5)
elasticregr.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticregr.predict(X_train)
y_preds_test = elasticregr.predict(X_test)

print("Best alpha value is: {}".format(elasticregr.alpha_))
print("R-squared of the model on the training set is: {}".format(elasticregr.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(elasticregr.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))



Best alpha value is: 0.1
R-squared of the model on the training set is: 0.764824683275236
-----Test set statistics-----
R-squared of the model on the test set is: 0.7757730201705589
Mean absolute error of the prediction is: 25079.27253351844
Mean squared error of the prediction is: 1505393589.1192317
Root mean squared error of the prediction is: 38799.40191703001
Mean absolute percentage error of the prediction is: 15.115559584164304


As we see, the R-squared of the model in the training set is 0.76; also, R-squared of the model in the test set is 0.78.


I think the ElasticNet regression model is the best one,because the R-squared higher than others.

These four different metrics are essentially using for the difference between what we know to be correct saleprice house and predicted one from the model. So, lower values are desirable for all four metrics. The lower the value, the better the performance of the model. So, in ElasticNet regression model values for all four metrics are lower than others.